<center><h1>Dekorator poprawiający funkcje typu `def (l=[]): pass`</h1></center>

In [ ]:
from inspect import signature, Parameter
from copy import deepcopy

def refresh(f):
    params_to_fix = {}
    sig = signature(f)
    positionals = 0
    for key, param in sig.parameters.items():
        if param.kind == Parameter.POSITIONAL_OR_KEYWORD and param.default is not param.empty:
            params_to_fix[key] = deepcopy(param.default)
            positionals = positionals + 1
        elif param.kind in (Parameter.POSITIONAL_ONLY, Parameter.POSITIONAL_OR_KEYWORD):
            positionals = positionals + 1

    checking_set = set(params_to_fix.keys())
    def decorator(*args, **kwargs):
        input_kwargs_set = set(kwargs.keys())
        if len(args) == positionals or input_kwargs_set >= checking_set:
            return f(*args, **kwargs)
        else:
            difference = checking_set - input_kwargs_set
            for key in difference:
                kwargs[key] = deepcopy(params_to_fix[key])
            return f(*args, **kwargs)
        
    return decorator
        

@refresh
def f(item, l=[]):
    l.append(item)
    return l

l = []
f(1)
f(2)